In [4]:
!pip install gradio
!pip install git+https://github.com/openai/whisper.git
!pip install transformers
!pip install TTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-n2tyd6ci
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-n2tyd6ci
  Resolved https://github.com/openai/whisper.git to commit f82bc59f5ea234d4b97fb2860842ed38519f7e65
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import whisper
import gradio as gr
import os
from pydub import AudioSegment
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from TTS.api import TTS
import torch

In [12]:
mname = "facebook/blenderbot-400M-distill"
chat_model = BlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

chat_model = chat_model.to(device)
SR_model = whisper.load_model("base")
SR_model = SR_model.to(device)

tts_model_name = TTS.list_models()[11]
tts = TTS(tts_model_name)

100%|███████████████████████████████████████| 139M/139M [00:13<00:00, 10.4MiB/s]


 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--tacotron2-DCA


100%|██████████| 339M/339M [00:11<00:00, 28.6MiB/s]


 > Model's license - MPL
 > Check https://www.mozilla.org/en-US/MPL/2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--multiband-melgan


100%|██████████| 82.8M/82.8M [00:01<00:00, 41.6MiB/s]


 > Model's license - MPL
 > Check https://www.mozilla.org/en-US/MPL/2.0/ for more info.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:/root/.local/share/tts/tts_models--en--ljspeech--tacotron2-DCA/scale_stats.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 2
 > Vocoder Model: multi

In [17]:
def querybot(audio_file):
    AudioSegment.from_wav(audio_file).export("file.mp3", format="mp3")
    UTTERANCE = SR_model.transcribe("file.mp3")["text"]

    inputs = tokenizer([UTTERANCE], return_tensors="pt").to(device)
    reply_ids = chat_model.generate(**inputs)
    response = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
    tts.tts_to_file(text=response, file_path='tts.mp3')

    output = response

    return output, 'tts.mp3'

In [18]:
gr.Interface(
    fn=querybot, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs=['text',gr.Audio(type='filepath')]).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://577b5b5a-a1df-4bb2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
